In [6]:
import torch
from torch import nn
from d2l import torch as d2l

In [7]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding), nn.ReLU(), # 第一个卷积层
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(), # 第二个conv 1*1卷积核 不改变通道数
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU()  # 第三个conv 1*1卷积核
    )

In [8]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0), # 此处为灰度图 channel=1
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(p=0.5), # y
    # 标签总共十个类别 - Fashion-MNIST
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d(output_size=(1,1)), # 全局平均池化层 高宽都变成1
    # 将四维的输出转成二维的输出，其形状为（批量大小，10）
    nn.Flatten()
)

In [ ]:
"""
(1,1,224,224)
nin_block: (224-11)/4+1=54
maxp: (54-3)/2+1=26
nin_block: (26+2*2-5+1)=26
mp:   (26-3)/2+1=12
nb:   12
mp:   5
dropout 5
nb:   384,5 -> 10,5
adaptive 10,5 -> 10,1 (取五个值的平均值)
"""

In [9]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())